In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline
from pyspark.sql import functions as fn, Row
from pyspark import sql
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression

import matplotlib.pyplot as plt
import pandas as pd

In [4]:
##############################################################################
##########       Spark creat and transform data             #################
##############################################################################
## Spark Method ##
Full_data = spark.read.csv('train_V2.csv', sep=',', inferSchema=True, header=True)
# seperate match type
update_fun = (fn.when(fn.col('matchType').contains('solo'), 'solo').when(fn.col('matchType').contains('duo' or 'crash'), 'duo')
                .otherwise('squad'))
Full_data = Full_data.withColumn('matchType', update_fun)
##############################################################################
###### collect solo match
New_df_solo = Full_data.filter(Full_data.matchType == 'solo')
columns = ['Id','groupId','matchId','matchType']
New_df_solo = New_df_solo.select([col for col in New_df_solo.columns if col not in columns])
pj_sp_df_solo = New_df_solo.sample(withReplacement=False, fraction=0.1, seed=3)
##############################################################################
###### collect team match
New_df_team  = Full_data.select('*').groupby('groupId').agg(fn.sum('damageDealt').alias('total_team_damage'),
                      fn.sum('kills').alias('total_team_kills'),
                      fn.sum('killPoints').alias('team_kill_points'),
                      fn.avg('killPlace').alias('team_kill_rank'),
                      fn.avg('rankPoints').alias('team_normal_rank'),
                      fn.sum('revives').alias('team_revives'),
                      fn.sum('boosts').alias('team_boosts'),
                      fn.sum('assists').alias('total_assists'),
                      fn.sum('DBNOs').alias('team_DBNOs'),
                      fn.sum(Full_data.rideDistance + Full_data.walkDistance + Full_data.swimDistance).alias('totalDistance'))
New_df_team  = New_df_team.join(Full_data,New_df_team .groupId == Full_data.groupId)


## delete damageDealt
columns = ['Id','groupId','matchId', 'roadKills','numGroups','rideDistance','walkDistance','swimDistance','kills','killPints','killPlace','rankPoints','revives','boosts','assists','DBNOs','damageDealt']
New_df_team = New_df_team.select([col for col in New_df_team.columns if col not in columns])

################################################################################
pj_sp_df_team = New_df_team.sample(withReplacement=False, fraction=0.1, seed=3)
##
#withColumn('solo',(fn.col('matchType') == 'solo').cast('int')).\
pj_sp_df_team = pj_sp_df_team.\
                    withColumn('duo',(fn.col('matchType') == 'duo').cast('int')).\
                    withColumn('squad',(fn.col('matchType') == 'squad').cast('int'))
pj_sp_df_team = pj_sp_df_team.drop('matchType')
pj_sp_df_team.show(5)
##############################################################################
##############################################################################
# Full Data if needed
#pj_sp_df_solo = New_df_solo
#pj_sp_df_team = New_df_team.\
#                    withColumn('duo',(fn.col('matchType') == 'duo').cast('int')).\
#                    withColumn('squad',(fn.col('matchType') == 'squad').cast('int'))
#pj_sp_df_team = pj_sp_df_team.drop('matchType')


Py4JJavaError: An error occurred while calling o91.csv.
: java.io.IOException: Failed to create local dir in C:\Users\Mingy\AppData\Local\Temp\blockmgr-0677d6dd-abaa-442d-901d-8d88a10f5610\0d.
	at org.apache.spark.storage.DiskBlockManager.getFile(DiskBlockManager.scala:70)
	at org.apache.spark.storage.DiskStore.remove(DiskStore.scala:117)
	at org.apache.spark.storage.BlockManager.removeBlockInternal(BlockManager.scala:1513)
	at org.apache.spark.storage.BlockManager.doPut(BlockManager.scala:1047)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:1085)
	at org.apache.spark.storage.BlockManager.putIterator(BlockManager.scala:843)
	at org.apache.spark.storage.BlockManager.putSingle(BlockManager.scala:1406)
	at org.apache.spark.broadcast.TorrentBroadcast.writeBlocks(TorrentBroadcast.scala:123)
	at org.apache.spark.broadcast.TorrentBroadcast.<init>(TorrentBroadcast.scala:88)
	at org.apache.spark.broadcast.TorrentBroadcastFactory.newBroadcast(TorrentBroadcas

In [ ]:
#######################################################
##########     Randrom Forest of Solo Data  ###########
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf_solo_df = pj_sp_df_solo
rf_solo_df.withColumnRenamed('winPlacePerc', 'label')
inputcols = rf_solo_df.columns
inputcols.remove('label')

#split data into train, validation and test
training_df, validation_df, testing_df = rf_solo_df.randomSplit([0.6, 0.3, 0.1], seed=0)

#stages
from pyspark.ml.feature import VectorAssembler
va1 = VectorAssembler().setInputCols(inputcols).setOutputCol('features')
rf1 = RandomForestRegressor(featuresCol = 'features')

# Chain indexer and forest in a Pipeline
rf_pipeline1 = Pipeline(stages=[va1, rf1])

# Train model.  This also runs the indexer.
rf_model1 = rf_pipeline1.fit(training_df)

#Test on validation dataset
predictions1 = rf_model.transform(validation_df)

#select example rows to display
predictions1.select('prediction', 'label','features').show(5)

# validation error
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions1)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse)

In [ ]:
#################################################################
##########     Randrom Forest of Team  Original Data  ###########

rf_team_df = pj_sp_df_team
rf_team_df.withColumnRenamed('winPlacePerc', 'label')
inputcols = rf_team_df.columns
inputcols.remove('label')

#split data into train, validation and test
training_df, validation_df, testing_df = rf_team_df.randomSplit([0.6, 0.3, 0.1], seed=0)

#stages
from pyspark.ml.feature import VectorAssembler
va2 = VectorAssembler().setInputCols(inputcols).setOutputCol('features')
rf2 = RandomForestRegressor(featuresCol = 'features')

# Chain indexer and forest in a Pipeline
rf_pipeline2 = Pipeline(stages=[va2, rf2])

# Train model.  This also runs the indexer.
rf_model2 = rf_pipeline2.fit(training_df)

#Test on validation dataset
predictions2 = rf_model.transform(validation_df)

#select example rows to display
predictions2.select('prediction', 'label','features').show(5)

# validation error
# Select (prediction, true label) and compute test error
rmse = evaluator.evaluate(predictions2)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse)